In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import os
import json

import torch
import pandas as pd
import numpy as np
pd.set_option("display.max_colwidth", 400)
import tqdm


from pytorch_pretrained_bert import modeling, tokenization
from language_modeling.runners import (
    tokenize_example, InputExample,
    convert_example_to_features, features_to_data,
)
from glue.wnli_utils import masking_predictor, compare_two_examples

device = torch.device("cpu")
bert_model_name = "bert-base-uncased"

model = modeling.BertForPreTraining.from_pretrained(bert_model_name)
model.to(device);
model.eval();

tokenizer = tokenization.BertTokenizer.from_pretrained(bert_model_name, do_lower_case=True)

04/23/2019 19:58:40 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /Users/thibault/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
04/23/2019 19:58:40 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file /Users/thibault/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /var/folders/qf/0zz0jmx54kg63r16jr2v1mq00000gn/T/tmpyj_mgrgm
04/23/2019 19:58:43 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "adapter_act": "gelu",
  "adapter_initializer_range": 0.0002,
  "adapter_size": 64,
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "int

In [17]:
FOLDER = "data/wnli"
WNLI_TRAIN_PATH = os.path.join(FOLDER, "train.tsv")
WNLI_DEV_PATH = os.path.join(FOLDER, "dev.tsv")
WNLI_TEST_PATH = os.path.join(FOLDER, "test.tsv")

train_df = pd.read_csv(WNLI_TRAIN_PATH, sep="\t")
val_df = pd.read_csv(WNLI_DEV_PATH, sep="\t")
test_df = pd.read_csv(WNLI_TEST_PATH, sep="\t")

In [19]:
STOP_AT = 10

train_pred_arr, train_all_alt_ls, pos_a, pos_b, tok_a, tok_b = masking_predictor(train_df.iloc[:STOP_AT], tokenizer, 
                                                                                 model, device)

# val_pred_arr, val_all_alt_ls, *_ = masking_predictor(val_df, tokenizer, model)

print("Train acc: ", (train_pred_arr==train_df["label"][:STOP_AT]).mean())
# print("Val acc: ", (val_pred_arr==val_df["label"]).mean())

print("mean train pred:", train_pred_arr.mean())
# print("mean val pred:", val_pred_arr.mean())


Train acc:  0.7
mean train pred: 0.6


# Original Winograd

In [7]:
path = "data/winograd/wsc273.json"    
data = pd.read_json(path)

In [20]:
len(data)

546

In [14]:
labels = []
preds = []
l = []
for i in tqdm.tqdm_notebook(range(data.question_id.max()+1)):
    sent1, sent2 = data[data.question_id == i].substitution
    labels.append(int(data[data.question_id == i].correctness.iloc[1]))
    pred, same_l = compare_two_examples(tokenizer, model, sent1, sent2, device, "masking")
    preds.append(pred)
    l.append(same_l)

In [12]:
labels = np.array(labels)
preds = np.array(preds)
l = np.array(l)
print((labels == preds).mean())

0.5384615384615384


In [13]:
labels.mean()

0.4981684981684982